# <center>Coursera Capstone Project for <br>IBM Data Science with Python</center>

#### <center>Lutz Wimmer</center>

## Table of Contents:
* [Assignment 1](#assignment1)
* [Assignment 2](#assignment2)
    * [Part 1](#part2.1)
    * [Part 2](#part2.2)
    * [Part 3](#part2.3)
* [Assignment 3](#assignment3)
    * [Part 1](#part3.1)
    * [Part 2](#part3.2)

## <br><br>Assignment 1 - Capstone Project Notebook <a class="anchor" id="assignment1"></a>


<br>This Notebook is part of the Capstone Project for the "IBM Data Science with Python" course on Coursera.

In [1]:
# import Python libraries
import pandas as pd
import numpy as np
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## <br><br>Assignment 2 - Segmenting and Clustering Neighborhoods in Toronto <a class="anchor" id="assignment2"></a>

### <br>Part 1 - Web scraping list of postal codes of Toronto, Canada <a class="anchor" id="part2.1"></a>

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# saving webpage to memory as text
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
webtext = requests.get(url).text
webpage = BeautifulSoup(webtext,"html5lib")

webtab = webpage.find("table",class_='wikitable sortable')

In [4]:
# creating pandas df
toronto = pd.DataFrame(columns=["PostalCode","Borough","Neighborhood"])

for row in webtab.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        pc = col[0].text.strip()
        bor = col[1].text.strip()
        nei = col[2].text.strip()
        toronto = toronto.append({"PostalCode": pc, "Borough": bor, "Neighborhood": nei}, ignore_index=True)

# filter unassigned values
fil = toronto.index[toronto['Borough'] == "Not assigned"].tolist()
toronto = toronto.drop(fil, axis = 0).reset_index(drop = True)

# checking for duplicate postal codes and "Not assigned" Neighborhoods
# toronto.iloc[:,0].duplicated().describe()
# toronto.iloc[:,2] == "Not assigned"
# none found
toronto

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
toronto.shape

(103, 3)

### <br>Part 2 - Assigning lat/long coordinates <a class="anchor" id="part2.2"></a>

Using the link to the .csv file, the geocoder doesn't work.

In [6]:
coords = pd.read_csv('https://cocl.us/Geospatial_data')
coords.shape

(103, 3)

In [7]:
# merging the two dataframes by PostalCode
coords.rename(columns={'Postal Code':'PostalCode'}, inplace = True)
toronto = toronto.merge(coords, on = 'PostalCode')
toronto

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


### <br>Part 3 - Clustering neighborhoods <a class="anchor" id="part2.3"></a>

using ".*Toronto*." to filter out any boroughs outside the city.

In [8]:
city = toronto[toronto['Borough'].str.match(".*Toronto*.")]
city['Borough'].unique()

array(['Downtown Toronto', 'East Toronto', 'West Toronto',
       'Central Toronto'], dtype=object)

The number of neighborhoods in the city is 39 and thus, the number of clusters is 39.<br>


In [9]:
import numpy as np
from sklearn.cluster import KMeans

# set number of clusters = number of neighborhoods
kclusters = len(city['Neighborhood'].unique())

toronto_clustering = city.drop(columns={'PostalCode','Borough','Neighborhood'})

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([22, 35, 32, 34, 12, 36, 11, 10,  4, 15, 33,  6, 16, 28, 23,  3, 37,
        8, 14,  9, 30, 20, 21, 25, 26,  1,  5, 17,  7, 19, 31,  2, 13, 24,
       18,  0, 38, 27, 29])


<br>Constructing a new dataframe with Toronto neighborhood centers coordinates for mapping.

In [10]:
neighborhoods = pd.DataFrame(columns=['Neighborhood','Cluster Labels','Latitude','Longitude']) 
names = city['Neighborhood'].unique()

for i in range(len(names)):
    one = names[i]
    two = kmeans.cluster_centers_[i][0]
    three = kmeans.cluster_centers_[i][1]
    four = kmeans.labels_[i]
    neighborhoods = neighborhoods.append({'Neighborhood':one,'Cluster Labels':four,'Latitude':two,'Longitude':three}, ignore_index=True)

neighborhoods.head()

,Neighborhood,Cluster Labels,Latitude,Longitude
0,"Regent Park, Harbourfront",22,43.667967,-79.367675
1,"Queen's Park, Ontario Provincial Government",35,43.648960,-79.456325
2,"Garden District, Ryerson",32,43.686412,-79.400049
3,St. James Town,34,43.668999,-79.315572
4,The Beaches,12,43.650571,-79.384568


Creating a map with folium.

In [11]:
# !conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
# create map
latitude = neighborhoods['Latitude'][3]
longitude = neighborhoods['Longitude'][3]
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Neighborhood'], neighborhoods['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

__A map with the Neighborhood centers for all 39 Neighborhoods in Toronto__

## <br><br>Assignment 3 - Financial Centers <a class="anchor" id="assignment3"></a>

### <br>Part 1 - Defining the problem <a class="anchor" id="part3.1"></a>

World financial centers are some of the most wealthy regions. They are spacially very concentrated and a hot spot for luxury products. In the aftermath of the coronavirus pandemic, thousands of restaurants shut down for good, even in the busy financial centers. But ones downfall is the others chance and people always need places to go and eat. __New restaurants are needed__, so you take your stimulus check and open a restaurant in __Frankfurt am Main__ (short: FFM), Germany. But what kind of restaurant? What does the international financial elite consume (apart from cocaine)? Is there a common type of restaurant between those cities that promises success? Let's dive into the data and find out!

### <br>Part 2 - The Data <a class="anchor" id="part3.2"></a>

We will compare data of the financial centers of the following Cities: __New York__, __Toronto__, __Tokyo__, __London__ and __Frankfurt__

In [13]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Adding _"&query=food"_ to the Foursquare API url to only get food-related venues. This way the search radius can be expanded without hitting the limit of 100 venues per city. The request and subsequent data frame building is looped over all 5 cities.

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# list of cities and their financial centers as coordinates
cities = ["New York", "Toronto", "Tokyo", "London", "Frankfurt"]
latitudes = [40.7077426113486, 43.64803026178278, 35.679822325779995, 51.515259192193064, 50.11081120228777]
longitudes = [-74.01157215146745, -79.3816696346114, 139.77014548424125, -0.08039578093893954, 8.67324827528688]

# Foursquare API credentials
CLIENT_ID = 'T0ETMOBTMRGKDWDG5VBSUFRUVWYJDTLHMLZHN2APZ545NNGR' # your Foursquare ID
CLIENT_SECRET = 'T03SSSFDVCM1JFVTX4YJLGS2IJPYD2AMRNWAR4DAXF2RFMZA' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 10000 # define radius

# a list to store all data frames in
all_df = list()

# loop over all 5 cities
for i in range(len(cities)):
# create URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query=food'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitudes[i], 
        longitudes[i], 
        radius, 
        LIMIT)

    results = requests.get(url).json()
    venues = results['response']['groups'][0]['items']
    nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
    filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
    nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
    nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
    nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    nearby_venues['city'] = cities[i]
# add city name to column
    all_df.append(nearby_venues)
    
all_df[0].head()

<ipython-input-14-5efed386b48f>:42: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng,city
0,Crown Shy,Restaurant,40.706187,-74.007490,New York
1,sweetgreen,Salad Place,40.705626,-74.008282,New York
2,Pisillo Italian Panini,Sandwich Place,40.710530,-74.007526,New York
3,Hudson Eats,Food Court,40.712666,-74.015901,New York
4,Los Tacos No. 1,Taco Place,40.714267,-74.008756,New York


<br>Although bistros and cafes also serve food, for simplicity, we only use restaurant-type categories.

In [15]:
# list restaurants data frames
restaurants = list()
for i in range(5):
    restaurants.append(all_df[i][all_df[i]['categories'].str.match(".*Restaurant*.")])
restaurants[1].head()

,name,categories,lat,lng,city
0,Canoe,Restaurant,43.647452,-79.381320,Toronto
1,Pai,Thai Restaurant,43.647923,-79.388579,Toronto
2,Richmond Station,American Restaurant,43.651569,-79.379266,Toronto
3,Byblos Toronto,Mediterranean Restaurant,43.647615,-79.388381,Toronto
4,The Keg Steakhouse + Bar - York Street,Restaurant,43.649987,-79.384103,Toronto


Some restaurants don't have a specification for their kitchen. These will be treated as "miscellaneous" category.

In [16]:
for i in range(5):
    print("Number of restaurants in",cities[i],":  ",len(restaurants[i]))
    print("Misc. Restaurants:", sum(restaurants[i]['categories'].str.match("Restaurant")) )

Number of restaurants in New York :   56
Misc. Restaurants: 1
Number of restaurants in Toronto :   53
Misc. Restaurants: 8
Number of restaurants in Tokyo :   66
Misc. Restaurants: 1
Number of restaurants in London :   55
Misc. Restaurants: 7
Number of restaurants in Frankfurt :   48
Misc. Restaurants: 5


In [17]:
# renaming "Restaurants" to "Miscellaneous Restaurant"
for i in range(5):
    restaurants[i].loc[:]['categories'] = restaurants[i].loc[:]['categories'].replace('Restaurant','Miscellaneous Restaurant')
restaurants[1].head()

,name,categories,lat,lng,city
0,Canoe,Miscellaneous Restaurant,43.647452,-79.381320,Toronto
1,Pai,Thai Restaurant,43.647923,-79.388579,Toronto
2,Richmond Station,American Restaurant,43.651569,-79.379266,Toronto
3,Byblos Toronto,Mediterranean Restaurant,43.647615,-79.388381,Toronto
4,The Keg Steakhouse + Bar - York Street,Miscellaneous Restaurant,43.649987,-79.384103,Toronto


<br>Now let's map the data.

In [18]:
map_all = folium.Map(location=[0, 0], zoom_start=2)

# add markers to map
for i in range(5):
    for lat, lng, name, category in zip(restaurants[i]['lat'], restaurants[i]['lng'], restaurants[i]['name'], restaurants[i]['categories']):
        label = '{}, {}'.format(name, category)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_all)  
    
map_all